In [1]:
from langchain_community.chat_models import ChatOllama
from typing import TypedDict

ollama_agent = ChatOllama(
    model="gpt-oss:20b",
    base_url="http://10.1.1.59:11434"
)


class State(TypedDict):
    query: str
    research: str
    summary: str
# Agent A: Research Agent
def research_node(state: State):
    print("-----------research node-----------------")
    q = state["query"]
    res = ollama_agent.invoke(f"Explain information about the following topic: {q}")
    state["research"] = res.content
    print(res.content)
    return state

# Agent B: Summarizer Agent
def summary_node(state: State):
    print("-----------summary node-----------------")
    text = state["research"]
    res = ollama_agent.invoke(f"Use Traditional Chinese. Summarize the following content into three key points: {text}")
    state["summary"] = res.content
    print(res.content)
    return state


from langgraph.graph import StateGraph, END

graph = StateGraph(State)

graph.add_node("ResearchAgent", research_node)
graph.add_node("SummarizerAgent", summary_node)

graph.set_entry_point("ResearchAgent")
graph.add_edge("ResearchAgent", "SummarizerAgent")
graph.add_edge("SummarizerAgent", END)

app = graph.compile()

resp = app.invoke({"query": "Ollama"})

print("-----------result-----------------")
print(resp["summary"])


/tmp/ipykernel_2739/424442926.py:4: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  ollama_agent = ChatOllama(


-----------research node-----------------
## Ollama – A Quick‑start Guide to the “Local‑LLM” Ecosystem

**What is Ollama?**  
Ollama is a lightweight, open‑source framework that lets you download, run, and manage large‑language‑models (LLMs) on your own hardware—no cloud connection required. Think of it as a “Model Hub + Runtime” for LLMs that lives on your laptop, desktop, or even a small edge device. It was launched in 2023 by **Ollama, Inc.** (the same folks behind the popular *Ollama* chat app that runs locally) and has quickly become a go‑to solution for developers, researchers, and hobbyists who want privacy‑first, low‑latency AI without paying per‑token fees.

---

### 1. The Big Picture

| Aspect | What Ollama Provides | Why It Matters |
|--------|---------------------|----------------|
| **Local inference** | Run any LLM on your GPU/CPU without a server | Keeps data on‑device, eliminates latency & cost |
| **Model hub** | Pre‑built, ready‑to‑use weights (Llama 3, Gemma, Mistra

In [2]:
from langgraph.graph import StateGraph, END
from langchain_community.chat_models import ChatOllama
from typing import TypedDict

ollama_agent = ChatOllama(
    model="gpt-oss:20b",
    base_url="http://10.1.1.59:11434"
)

ROUTER_PROMPT = """
你是一個問題分流 Agent。  
你的任務是根據使用者的輸入，判斷應該由哪個專家 Agent 處理。  
請只輸出對應的 Agent 名稱，不要輸出多餘文字。  

可選的 Agent 類別有：  
- TutorialAgent → 安裝、環境設定、入門指引  
- SyntaxCoachAgent → Go 語法、資料型態、關鍵字解釋（例如 :=, defer, map, struct, interface）  
- PracticalExampleAgent → REST API、測試、範例程式碼、模組應用  
- ConcurrencyExpertAgent → goroutines、channels、select、同步/非同步程式設計  
- TroubleshootingAgent → 錯誤訊息診斷、編譯問題、模組管理錯誤  
- KnowledgeTreeAgent → 系統化知識結構、學習路徑、知識樹  

輸出格式（JSON）：  
{
  "agent": "<AgentName>"
}
"""

def route_question(user_question: str) -> str:
    """RouterAgent: 分派使用者的問題到對應的 Agent"""
    
    prompt = ROUTER_PROMPT + f"\n\n使用者的輸入是：{user_question}"
    # print(prompt)

    response = ollama_agent.invoke(prompt)   # 用 langchain_community ChatOllama
    output = response.content.strip()

    return output

if __name__ == "__main__":
    test_questions = [
        "Go 裡的 := 是什麼意思？",
        "幫我寫一個 Go REST API 範例",
        "我安裝 Go 之後執行 go run 報錯",
        "請幫我規劃一份學習 Golang 的知識架構",
    ]

    for q in test_questions:
        agent = route_question(q)
        print(f"Q: {q}\n→ 分配給: {agent}\n")
# graph = StateGraph()

# # 節點定義
# graph.add_node("RouterAgent", router_agent)
# graph.add_node("TutorialAgent", tutorial_agent)
# graph.add_node("SyntaxCoachAgent", syntax_coach_agent)
# graph.add_node("PracticalExampleAgent", practical_example_agent)
# graph.add_node("ConcurrencyExpertAgent", concurrency_expert_agent)
# graph.add_node("TroubleshootingAgent", troubleshooting_agent)
# graph.add_node("KnowledgeTreeAgent", knowledge_tree_agent)
# graph.add_node("AnswerCollector", answer_collector)

# # 邊 (Router → 對應 Agent)
# graph.add_edge("RouterAgent", "TutorialAgent")
# graph.add_edge("RouterAgent", "SyntaxCoachAgent")
# graph.add_edge("RouterAgent", "PracticalExampleAgent")
# graph.add_edge("RouterAgent", "ConcurrencyExpertAgent")
# graph.add_edge("RouterAgent", "TroubleshootingAgent")
# graph.add_edge("RouterAgent", "KnowledgeTreeAgent")

# # Agent → AnswerCollector
# for agent in [
#     "TutorialAgent", "SyntaxCoachAgent", "PracticalExampleAgent",
#     "ConcurrencyExpertAgent", "TroubleshootingAgent", "KnowledgeTreeAgent"
# ]:
#     graph.add_edge(agent, "AnswerCollector")

# # AnswerCollector → END
# graph.add_edge("AnswerCollector", END)

# # 入口
# graph.set_entry_point("RouterAgent")

# # 建立應用
# app = graph.compile()

Q: Go 裡的 := 是什麼意思？
→ 分配給: {
  "agent": "SyntaxCoachAgent"
}

Q: 幫我寫一個 Go REST API 範例
→ 分配給: {
  "agent": "PracticalExampleAgent"
}

Q: 我安裝 Go 之後執行 go run 報錯
→ 分配給: {
  "agent": "TroubleshootingAgent"
}

Q: 請幫我規劃一份學習 Golang 的知識架構
→ 分配給: {
  "agent": "KnowledgeTreeAgent"
}



In [ ]:
from __future__ import annotations
from typing import TypedDict, Dict, Any
from pathlib import Path

# LangChain - Ollama
try:
    from langchain_ollama import ChatOllama
except ImportError:
    from langchain_community.chat_models import ChatOllama  # 舊版用這個

from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

# LangGraph
from langgraph.graph import StateGraph, END


# ------------------------
# 載入本地 Prompt
# ------------------------
PROMPT_FILES = {
    "DevEnvAgent": Path("DevEnvAgent.txt"),
    "IssueAgent": Path("IssueAgent.txt"),
    "CodeTraceAgent": Path("CodeTraceAgent.txt"),
    "TestingAgent": Path("TestingAgent.txt"),
}

PROMPTS: Dict[str, str] = {k: v.read_text(encoding="utf-8") for k, v in PROMPT_FILES.items()}


# ------------------------
# LLM (Ollama)
# ------------------------
llm = ChatOllama(
    base_url="http://10.1.1.59:11434",   # 確保 ollama 服務在本機 11434
    model="gpt-oss:20b",   # 換成你本地可用的模型
    temperature=0.1,
)


# ------------------------
# Agent 工廠
# ------------------------
def make_agent(system_prompt: str):
    # 用 f-string 把文件內容塞進 system，不讓 LangChain 解析裡面的 {}
    template = """{system_prompt}

使用者的問題如下：
{user_input}
"""
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt.partial(system_prompt=system_prompt.strip()) | llm | StrOutputParser()
    return chain

AGENTS = {name: make_agent(prompt) for name, prompt in PROMPTS.items()}


# RouterAgent Prompt
ROUTER_PROMPT = """
你是一個問題分流 Agent。  
你的任務是根據使用者的輸入，判斷應該交給哪個專家 Agent 處理。  
請只輸出對應的 Agent 名稱，不要輸出多餘文字。  

可選的 Agent 類別有：
- DevEnvAgent → 開發環境、VSCode、Dev Container、iptables、dockerd、debug mode
- IssueAgent → GitHub Issue、Swarm 問題、錯誤訊息診斷
- CodeTraceAgent → Code Trace、Container 啟動邏輯、containerd、gRPC
- TestingAgent → 測試、unit test、integration test、TESTFLAGS、bridge_test.go

輸出格式：
<AgentName>
"""

# 建立 Prompt 模板
router_prompt = ChatPromptTemplate.from_messages([
    ("system", ROUTER_PROMPT),
    ("human", "{question}")
])

router_chain = router_prompt | llm

def route_question(user_question: str) -> str:
    """呼叫 LLM 來判斷要分派的 Agent"""
    response = router_chain.invoke({"question": user_question})
    output = response.content
    print("=================route_question")
    print(response)
    print("=================route_question")
    return output
 

# ------------------------
# LangGraph 狀態定義
# ------------------------
class GraphState(TypedDict):
    user_input: str
    routed: str | None
    response: str


# ------------------------
# Graph 節點
# ------------------------
def router_node(state: GraphState) -> GraphState:
    agent_name = route_question(state["user_input"])
    print(agent_name)
    return {**state, "routed": agent_name}

def agent_node(state: GraphState) -> GraphState:
    agent_name = state["routed"]
    chain = AGENTS[agent_name]
    output = chain.invoke({"user_input": state["user_input"]})
    return {**state, "response": output}


# ------------------------
# 建立 Graph
# ------------------------
graph = StateGraph(GraphState)
graph.add_node("router", router_node)
graph.add_node("agent", agent_node)

graph.set_entry_point("router")
graph.add_edge("router", "agent")
graph.add_edge("agent", END)

app = graph.compile()


# ------------------------
# 測試用函數
# ------------------------
def run_once(user_text: str) -> Dict[str, Any]:
    state = {"user_input": user_text, "routed": None, "response": ""}
    result = app.invoke(state)
    return {
        "routed_agent": result["routed"],
        "response": result["response"]
    }


# ------------------------
# CLI 互動
# ------------------------
if __name__ == "__main__":
    print("=== RouterAgent + 4 Agents (Ollama) ===")
    while True:
        q = input("\nYou > ").strip()
        if q.lower() in {"exit", "quit"}:
            break
        out = run_once(q)
        print(f"[Router → {out['routed_agent']}]")
        print(out["response"])

=== RouterAgent + 4 Agents (Ollama) ===



You >  run vscode


=================route_question
content='DevEnvAgent' additional_kwargs={} response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-09-14T13:17:31.399820097Z', 'done': True, 'done_reason': 'stop', 'total_duration': 619140148, 'load_duration': 82524262, 'prompt_eval_count': 243, 'prompt_eval_duration': 29764787, 'eval_count': 33, 'eval_duration': 504725680, 'model_name': 'gpt-oss:20b'} id='run--9dfbca75-e637-4861-9823-d5a044618965-0' usage_metadata={'input_tokens': 243, 'output_tokens': 33, 'total_tokens': 276}
=================route_question
DevEnvAgent
[Router → DevEnvAgent]
以下是使用 VSCode 來啟動並除錯 Moby 專案的完整流程（包含 Dev Container 與 Debug 模式）：

---

## 1️⃣ 先決條件

| 項目 | 需求 | 參考 |
|------|------|------|
| VSCode | 版本 1.70+ |  |
| VSCode 插件 | **Dev Containers** |  |
| Docker | 安裝在宿主機（VM）上 |  |
| 目錄 | 你已經 clone 下來的 `moby` 專案 |  |

> **提示**：如果你還沒安裝 Docker，請先在宿主機上安裝並確認 `docker run hello-world` 能正常執行。

---

## 2️⃣ 在 VSCode 內部啟動 Dev Container

1. **打開 VSCode**  
   - 進入你 clone 下來的 `moby` 專案資料夾


You >  如何填寫launch.json in vscode, launch.json only


=================route_question
content='DevEnvAgent' additional_kwargs={} response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-09-14T13:18:33.836950809Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1268998446, 'load_duration': 82414253, 'prompt_eval_count': 252, 'prompt_eval_duration': 126462759, 'eval_count': 68, 'eval_duration': 1058099943, 'model_name': 'gpt-oss:20b'} id='run--36f1b7c0-8367-4b68-8331-02801b8a086f-0' usage_metadata={'input_tokens': 252, 'output_tokens': 68, 'total_tokens': 320}
=================route_question
DevEnvAgent
[Router → DevEnvAgent]
```json
{
    "version": "0.2.0",
    "configurations": [
        {
            "name": "Launch Package",
            "type": "go",
            "request": "launch",
            "mode": "auto",
            "program": "${fileDirname}"
        }
    ]
}
```


...

=================route_question
content='DevEnvAgent' additional_kwargs={} response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-09-14T13:43:18.405644041Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1579931006, 'load_duration': 83032606, 'prompt_eval_count': 250, 'prompt_eval_duration': 122727509, 'eval_count': 88, 'eval_duration': 1371472302, 'model_name': 'gpt-oss:20b'} id='run--0c576023-da99-42cd-9c4a-e83386c53b2e-0' usage_metadata={'input_tokens': 250, 'output_tokens': 88, 'total_tokens': 338}
=================route_question
DevEnvAgent
[Router → DevEnvAgent]
以下是你可以直接照著執行的步驟，讓 Moby Project 在 VSCode 內以 **debug mode** 執行。  
（所有資訊均來自你提供的文件，請依照順序操作，若遇到錯誤請參考對應的「可能錯誤」說明。）

---

## 1️⃣ 先確定開發環境已安裝好

| 步驟 | 內容 | 參考 |
|------|------|------|
| 1 | **安裝 VSCode**（若尚未安裝） |  |
| 2 | **安裝 VSCode 插件：Dev Containers** |  |
| 3 | **重啟 VSCode**，確認畫面顯示「container bash」 |  |
| 4 | **在 VSCode 指令面板（Ctrl+Shift+P）輸入** `Reopen in Container` |  |

> 這一步會把整個專案打包進一個 Docker 容器，並在容器內開啟 VSCode。


You >  what is moby project


=================route_question
content='DevEnvAgent' additional_kwargs={} response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-09-14T13:51:52.062943721Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5716286111, 'load_duration': 2411111516, 'prompt_eval_count': 246, 'prompt_eval_duration': 256729838, 'eval_count': 192, 'eval_duration': 3012339197, 'model_name': 'gpt-oss:20b'} id='run--6f85672c-a19e-4676-afbb-fc7c6458eed7-0' usage_metadata={'input_tokens': 246, 'output_tokens': 192, 'total_tokens': 438}
=================route_question
DevEnvAgent
[Router → DevEnvAgent]
**Moby Project**  
The Moby Project is the open‑source foundation that Docker is built on.  
Its source code and contribution guidelines are hosted on GitHub:

- Repository: <https://github.com/moby/moby>  
- Contribution docs: <https://github.com/moby/moby/tree/master/docs/contributing>  

The project contains the core components that make up Docker, and developers can contribute to it following the 


You >  怎麼安裝moby


=================route_question
content='DevEnvAgent' additional_kwargs={} response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-09-14T13:53:05.50495072Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1646232821, 'load_duration': 84316729, 'prompt_eval_count': 247, 'prompt_eval_duration': 123941589, 'eval_count': 92, 'eval_duration': 1435741526, 'model_name': 'gpt-oss:20b'} id='run--a4b23224-cced-453f-a224-ad63ceb40eeb-0' usage_metadata={'input_tokens': 247, 'output_tokens': 92, 'total_tokens': 339}
=================route_question
DevEnvAgent
[Router → DevEnvAgent]
## 如何安裝 Moby

以下列出最常用的安裝方式，僅依照你提供的文件內容。

---

### 1. 直接在宿主機（VM）上安裝

1. **安裝 Docker**  
   在宿主機（例如 Rocky Linux 8）上先安裝 Docker，這是執行 Moby 的前置條件。  
   ```bash
   # 以 Rocky Linux 8 為例
   sudo dnf install -y docker
   sudo systemctl enable --now docker
   ```

2. **克隆 Moby 專案**  
   ```bash
   git clone https://github.com/moby/moby
   cd moby
   ```

3. **切換到你想要的分支或建立新分支**  
   ```bash
   git checkout -b dry-run-